---
# 환경 설정
---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
!pip install mxnet
!pip install gluonnlp tqdm
!pip install sentencepiece
!pip install transformers==4.3.0
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 4.0 MB/s 
     |████████████████████████████████| 3.3 MB 38.3 MB/s 
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc2
    Uninstalling tokenizers-0.8.1rc2:
      Successfully uninstalled tokenizers-0.8.1rc2
  Attempting uninstall: transformers
    Found existing installation: transformers 3.3.0
    Uninstalling transformers-3.3.0:
      Successfully uninstalled transformers-3.3.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# !git clone https://github.com/SKTBrain/KoBERT.git

In [26]:
cd /content/drive/MyDrive/aiffel/NLP/KoBERT

/content/drive/MyDrive/aiffel/NLP/KoBERT


In [ ]:
!pip install -r requirements.txt

In [ ]:
pip install wandb

In [ ]:
# pip install transformers-interpret

In [7]:
import wandb

wandb.login()
wandb.init()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: glad415. Use `wandb login --relogin` to force relogin


In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [3]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import re
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

In [35]:
#kobert
# from kobert.utils import get_tokenizer
# from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [27]:
# GPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/aiffel/AIFFELTON/TAPT_Model_Save")

bertmodel = AutoModel.from_pretrained("/content/drive/MyDrive/aiffel/AIFFELTON/TAPT_Model_Save")

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/aiffel/AIFFELTON/TAPT_Model_Save and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---
# 학습용 데이터셋 전처리
---

In [ ]:
# data2 = pd.read_csv('/content/drive/MyDrive/aiffel/AIFFELTON/for_model_labeling_folder/append_dataset.csv')
# del data2['Unnamed: 0']
# data2[['taste','quantity','delivery']] = 0
# data2.columns = ['review','taste','quantity','delivery']
# data2

,review,taste,quantity,delivery
0,아따 콧수염 아저씨 싸움 잘하네,0,0,0
1,좋다 누가 뭐래도 난 좋았다,0,0,0
2,노잼 뒤처리안한느낌의 찝찝함과 쓸대없이너무짧은 러닝타임,0,0,0
3,강수연씨 팬이라 보게된작품인데 정말 오랜만에 깊에 몰입해서 감상했던 영화네요,0,0,0
4,보는내내 흥미진진 반전도 예상 못했음,0,0,0
...,...,...,...,...
2995,넘 질질끄는듯 양수빈은 007인듯 권선징악 이면 좀 빨리전개되길 넘지루해,0,0,0
2996,대한민국은 이렇게 되질 않길 빈다,0,0,0
2997,이렇게 베베 꼬아놓아도 됩니까,0,0,0
2998,오래간만에 나타난 한국 애니메이션의 가능성,0,0,0


In [39]:
train_data = pd.read_csv('/content/drive/MyDrive/aiffel/AIFFELTON/for_model_labeling_folder/only_train.csv')
train_data = train_data[train_data['dataset'] == 'crawlling']

In [40]:
train_data.columns = ['review','dataset','taste','delivery','quantity']
train_data = train_data[['review','dataset','taste','quantity','delivery']]
train_data = train_data.reset_index(drop = True)
train_data

,review,dataset,taste,quantity,delivery
0,현생인류가 감당하기에는 110만 년은 이른 기술력이다 이것이 정교함 이것이 집중력 ...,crawlling,0,0,0
1,소스도 듬뿍 맛도 있었어요,crawlling,1,0,0
2,맛있어요 돈까스덮밥이 좋네요,crawlling,1,0,0
3,맛있게 잘먹었어요 근데 쫄면사이즈업이맞나요,crawlling,1,0,0
4,딱보통이었음 치킨이아쉽,crawlling,0,0,0
...,...,...,...,...,...
31204,맛잇게 잘먹엇어용 ㅎ,crawlling,1,0,0
31205,호식이 두마리 치킨은 개인적으로 매운 간장치킨이 젤 맛있는거 같아요 앞으로도 쭉 시...,crawlling,1,0,0
31206,오늘도 잘먹었습니당,crawlling,0,0,0
31207,빅맥은 역시 사랑이네요,crawlling,0,0,0


In [41]:
test_data = pd.read_csv('/content/drive/MyDrive/aiffel/AIFFELTON/for_model_labeling_folder/only_bench.csv')
test_data

,review,taste_label,deliver_label,quantity_label
0,15분만에 왔네요 배달도 빠르고,0,1,0
1,1955더블 맛있네요 배달이 좀 오래걸렸어요,1,1,0
2,1시간 넘게 기다렸고 그래서 식어서 왔지만 그래도 맛있었으니 됐습니다,1,1,0
3,1시간지남 아직 안옴 그런데 후기작성하라고 메세지 옴,0,1,0
4,1인분으로 딱적당해요 많지도 적지도않은,0,0,1
...,...,...,...,...
4638,흐으 참 맛있습니다,1,0,0
4639,흔들리지 않게 잘 고정돼서 와서 좋았구,0,1,0
4640,흠 개인의취향 다른메뉴는 좋은데 이메뉴는 너무 퍽퍽한 맛을 낸다는 제 개인적인 입맛...,1,0,0
4641,흠 빅맥보단 치킨버거가 괜찬은듯,0,0,0


In [42]:
test_data.columns = ['review','taste','delivery','quantity']
test_data = test_data[['review','taste','quantity','delivery']]
test_data

,review,taste,quantity,delivery
0,15분만에 왔네요 배달도 빠르고,0,0,1
1,1955더블 맛있네요 배달이 좀 오래걸렸어요,1,0,1
2,1시간 넘게 기다렸고 그래서 식어서 왔지만 그래도 맛있었으니 됐습니다,1,0,1
3,1시간지남 아직 안옴 그런데 후기작성하라고 메세지 옴,0,0,1
4,1인분으로 딱적당해요 많지도 적지도않은,0,1,0
...,...,...,...,...
4638,흐으 참 맛있습니다,1,0,0
4639,흔들리지 않게 잘 고정돼서 와서 좋았구,0,0,1
4640,흠 개인의취향 다른메뉴는 좋은데 이메뉴는 너무 퍽퍽한 맛을 낸다는 제 개인적인 입맛...,1,0,0
4641,흠 빅맥보단 치킨버거가 괜찬은듯,0,0,0


In [43]:
# del train_data['dataset']
# train_data = pd.concat([train_data, data2], ignore_index = True)
# train_data

In [44]:
# train_data = train_data.dropna()

In [45]:
# train_data = pd.read_csv('/content/drive/MyDrive/aiffel/AIFFELTON/for_model_labeling_folder/Final_Train_dataset.csv')
# del train_data['Unnamed: 0']
# test_data = pd.read_csv('/content/drive/MyDrive/aiffel/AIFFELTON/for_model_labeling_folder/Fianl_Test_dataset.csv')
# test_data

In [46]:
import numpy as np 

total_data_text = list(train_data['review'])
# 텍스트데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,  
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 문장의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens) / len(num_tokens)))

문장길이 평균 :  35.76532410522606
문장길이 최대 :  250
문장길이 표준편차 :  34.5032704473211
pad_sequences maxlen :  104
전체 문장의 0.9479316863725208%가 maxlen 설정값 이내에 포함됩니다. 


---
# 모델
---

In [6]:
class BERTDataset(Dataset): 
  def __init__(self, dataset, bert_tokenizer, max_len):
    
    self.sentences = [bert_tokenizer(i, truncation=True, return_token_type_ids=True, padding="max_length",
                                     max_length = max_len, return_tensors = 'pt') for i in dataset['review']]
    self.max_len = max_len
    self.taste = [np.int64(i) for i in dataset['taste']]
    self.quantity = [np.int64(i) for i in dataset['quantity']]
    self.delivery = [np.int64(i) for i in dataset['delivery']]
  
  def __getitem__(self,i):
    self.input_ids = self.sentences[i]['input_ids']
    self.attention_mask_token = self.sentences[i]['attention_mask']
    self.token_type_ids = self.sentences[i]['token_type_ids']

    return self.input_ids, self.attention_mask_token, self.token_type_ids, self.taste[i], self.quantity[i], self.delivery[i]

  def __len__(self):
    return (len(self.taste))

In [7]:
# 하이퍼파라미터 세팅
max_len = 104
batch_size = 32
warmup_ratio = 0.1
num_epochs = 1
max_grad_norm = 1
log_interval = 300
learning_rate =  5e-7

In [49]:
data_train = BERTDataset(train_data, tokenizer, max_len)
data_test = BERTDataset(test_data, tokenizer, max_len)

In [50]:
train_dataloader = torch.utils.data.DataLoader(data_train, shuffle=True, batch_size=batch_size, num_workers=3, drop_last=True)
test_dataloader = torch.utils.data.DataLoader(data_test, shuffle=False, batch_size=batch_size, num_workers=3, drop_last=True)

In [ ]:
# dataloader test 
for a,b,c,d,e,f in train_dataloader:
  print(a)
  print(b)
  print(c)
  print(d)
  print(e)
  print(f)
  print('='*100)
  print('input_ids: ',a)
  print('attention_mask: ',b.shape)
  print('token_type_ids: ',c.shape)
  print('taste_label: ',d.shape)
  break


In [ ]:
# wandb.init(project = 'klue_bert_TAPT_before',
#                  entity="glad415",
#                  config = {
#                      "hidden_size" : 768,
#                      "max_len" : max_len,
#                      "batch_size" : batch_size,
#                      "learning_rate" : learning_rate,
#                      "epochs" : num_epochs,
#                      "optimizer" : 'adamW',
#                      "loss" : "BCEloss",
#                  })

In [8]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 ################# 클래스 수 조정 ##################
                 num_classes=1,   
                 dr_rate=0.3,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.taste = nn.Linear(hidden_size , num_classes)
        self.quantity = nn.Linear(hidden_size , num_classes)
        self.delivery = nn.Linear(hidden_size , num_classes)

        self.dropout = nn.Dropout(p=dr_rate)

        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input_ids, attention_mask, token_type_ids):

        out = self.bert(input_ids = input_ids.long(), token_type_ids = token_type_ids.long(), attention_mask = attention_mask.to(device))
        out = self.dropout(out.pooler_output)

        taste_out = self.taste(out)
        quantity_out = self.quantity(out)
        delivery_out = self.delivery(out)   

        return self.sigmoid(taste_out), self.sigmoid(quantity_out), self.sigmoid(delivery_out)
    
    ############################# Frezee #############################
    # def freeze_bert_encoder(self):
    #     for param in self.bert.parameters():
    #         param.requires_grad = False
    
    # def unfreeze_bert_encoder(self):
    #     for param in self.bert.parameters():
    #         param.requires_grad = True
    ##################################################################

In [53]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.3).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.BCELoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [54]:
for e in range(num_epochs):
    train_acc = 0.0
    total_f1 = 0.0
    for batch_id, (input_ids, attention_mask_token, token_type_ids, taste_label, quantity_label, delivery_label) in enumerate(tqdm_notebook(train_dataloader)):
        
        optimizer.zero_grad()
        
        input_ids = input_ids.long().to(device)
        input_ids = input_ids.squeeze(1)

        token_type_ids = token_type_ids.long().to(device)
        token_type_ids = token_type_ids.squeeze(1)
        
        attention_mask_token= attention_mask_token.squeeze(1)

        taste_label = taste_label.float().to(device)
        taste_label = taste_label.unsqueeze(1)

        quantity_label = quantity_label.float().to(device)
        quantity_label = quantity_label.unsqueeze(1)

        delivery_label = delivery_label.float().to(device)
        delivery_label = delivery_label.unsqueeze(1)

        taste_out, quantity_out, delivery_out = model(input_ids, attention_mask_token, token_type_ids)

        # Loss
        ################################################################################################
        taste_loss = loss_fn(taste_out, taste_label)
        quantity_loss = loss_fn(quantity_out, quantity_label)
        delivery_loss = loss_fn(delivery_out, delivery_label)

        total_loss = (taste_loss + quantity_loss + delivery_loss)
        total_loss.backward() 
        ################################################################################################

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        # Accuracy
        ################################################################################################
        taste_preds = (taste_out >= 0.5).float()
        quantity_preds = (quantity_out >= 0.5).float()
        delivery_preds = (delivery_out >= 0.5).float()

        taste_accuracy = (taste_preds == taste_label).cpu().numpy().mean()
        quantity_accuracy = (quantity_preds == quantity_label).cpu().numpy().mean()
        delivery_accuracy = (delivery_preds == delivery_label).cpu().numpy().mean()

        train_acc += (taste_accuracy + quantity_accuracy + delivery_accuracy) / 3
        ################################################################################################

        # F1_Score
        ################################################################################################
        taste_f1 = f1_score(taste_preds.cpu().data, taste_label.cpu(), average = 'micro')
        quantity_f1 = f1_score(quantity_preds.cpu().data, quantity_label.cpu(), average = 'micro')
        delivery_f1 = f1_score(delivery_preds.cpu().data, delivery_label.cpu(), average = 'micro')
        
        total_f1 +=  (taste_f1 + quantity_f1 + delivery_f1) / 3
        ################################################################################################

        if batch_id % log_interval == 0:

            print(f"epoch {e+1} batch id {batch_id+1} loss {total_loss.data.cpu().numpy():.3f} train acc {train_acc / (batch_id+1):.3f}")
            print(f'taste_accuracy {taste_accuracy:.3f}  | quantity_accuracy {quantity_accuracy:.3f}  | delivery_accuracy {delivery_accuracy:.3f}')
            print(f'f1_score:{total_f1 / (batch_id+1):.3f}')
            print('#'*80)

            # WandB
            #############################################################################################################################
            # wandb.log({'interval_Loss': total_loss.data.cpu().numpy(),
            #             'interval_taste_Accuracy' :taste_accuracy, 'interval_quantity_Accuracy' : quantity_accuracy, 'interval_delivery_Accuracy' : delivery_accuracy,
            #             'interval_taste_F1' : taste_f1, 'interval_quantity_F1' : quantity_f1, 'interval_delivery_F1' : delivery_f1})
            #############################################################################################################################

    print(f"epoch {e+1} epoch_Train_Accuracy {train_acc / (batch_id+1):.3f}, epoch_F1_score {total_f1 / (batch_id+1):.3f}")
    
    # WandB
    #############################################################################################################################
    # wandb.log({'epoch_Train_Accuracy' : train_acc / (batch_id+1), 'epoch_F1_score' : total_f1 / (batch_id+1)})
    #############################################################################################################################

    model.eval()
    test_f1 = 0.0
    test_acc = 0.0
    for batch_id, (input_ids, attention_mask_token, token_type_ids, taste_label, quantity_label, delivery_label) in enumerate(tqdm_notebook(test_dataloader)):
        
        input_ids = input_ids.long().to(device)
        input_ids = input_ids.squeeze(1)

        token_type_ids = token_type_ids.long().to(device)
        token_type_ids = token_type_ids.squeeze(1)
        
        attention_mask_token= attention_mask_token.squeeze(1)

        taste_label = taste_label.float().to(device)
        taste_label = taste_label.unsqueeze(1)

        quantity_label = quantity_label.float().to(device)
        quantity_label = quantity_label.unsqueeze(1)

        delivery_label = delivery_label.float().to(device)
        delivery_label = delivery_label.unsqueeze(1)

        taste_out, quantity_out, delivery_out = model(input_ids, attention_mask_token, token_type_ids)

        # Test_Accuracy
        ################################################################################################    
        taste_preds = (taste_out >= 0.5).float()
        quantity_preds = (quantity_out >= 0.5).float()
        delivery_preds = (delivery_out >= 0.5).float()

        taste_accuracy = (taste_preds == taste_label).cpu().numpy().mean()
        quantity_accuracy = (quantity_preds == quantity_label).cpu().numpy().mean()
        delivery_accuracy = (delivery_preds == delivery_label).cpu().numpy().mean()

        test_acc += (taste_accuracy + quantity_accuracy + delivery_accuracy) / 3
        ################################################################################################ 

        # Test_F1_Score
        ################################################################################################
        taste_f1 = f1_score(taste_preds.cpu().data, taste_label.cpu(), average = 'micro')
        quantity_f1 = f1_score(quantity_preds.cpu().data, quantity_label.cpu(), average = 'micro')
        delivery_f1 = f1_score(delivery_preds.cpu().data, delivery_label.cpu(), average = 'micro')
        
        test_f1 +=  (taste_f1 + quantity_f1 + delivery_f1) / 3
        ################################################################################################

    print('-'*40,'Test_Value','-'*40)
    print(f"test_Accuracy {test_acc / (batch_id+1):.3f}, test_F1_score {test_f1/ (batch_id+1):.3f}")
    print('-'*92)
    # wandb.log({'Test_acc' : test_acc / (batch_id+1), 'Test_F1_Score' : test_f1/ (batch_id+1)})

  0%|          | 0/975 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.318 train acc 0.333
taste_accuracy 0.375  | quantity_accuracy 0.406  | delivery_accuracy 0.219
f1_score:0.333
################################################################################
epoch 1 batch id 301 loss 1.571 train acc 0.656
taste_accuracy 0.719  | quantity_accuracy 0.812  | delivery_accuracy 0.688
f1_score:0.656
################################################################################
epoch 1 batch id 601 loss 1.212 train acc 0.730
taste_accuracy 0.750  | quantity_accuracy 0.844  | delivery_accuracy 0.875
f1_score:0.730
################################################################################
epoch 1 batch id 901 loss 1.007 train acc 0.773
taste_accuracy 0.781  | quantity_accuracy 0.844  | delivery_accuracy 0.938
f1_score:0.773
################################################################################
epoch 1 epoch_Train_Accuracy 0.780, epoch_F1_score 0.780


  0%|          | 0/145 [00:00<?, ?it/s]

---------------------------------------- Test_Value ----------------------------------------
test_Accuracy 0.819, test_F1_score 0.819
--------------------------------------------------------------------------------------------


In [71]:
# wandb.finish()

NameError: ignored

In [55]:
# torch.save(model.state_dict(), '/content/drive/MyDrive/aiffel/AIFFELTON/model_save/koBERT_Multi_label_0913_dict.pt')

In [15]:
device = torch.device("cpu")

In [28]:
model = BERTClassifier(bertmodel).to(device)

In [29]:
model.load_state_dict(torch.load('/content/drive/MyDrive/aiffel/AIFFELTON/model_save/koBERT_Multi_label_0913_dict.pt', map_location=device))

<All keys matched successfully>

In [30]:
def predict(predict_sentence):

    dataset_another = pd.DataFrame({'review': predict_sentence, 'taste' : 0 , 'quantity' : 0, 'delivery' : 0}, index = [0])

    another_test = BERTDataset(dataset_another,tokenizer, max_len)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=1, num_workers=5)
    
    model.eval()

    for batch_id, (input_ids, attention_mask_token, token_type_ids, taste_label, quantity_label, delivery_label) in enumerate(test_dataloader):
            
        input_ids = input_ids.long().to(device)
        input_ids = input_ids.squeeze(1)

        token_type_ids = token_type_ids.long().to(device)
        token_type_ids = token_type_ids.squeeze(1)
        
        attention_mask_token= attention_mask_token.squeeze(1)

        taste_label = taste_label.long().to(device)
        quantity_label = quantity_label.long().to(device)
        delivery_label = delivery_label.long().to(device)

        taste_out, quantity_out, delivery_out = model(input_ids, attention_mask_token, token_type_ids)

        for taste, quantity, delivery in zip(taste_out, quantity_out, delivery_out):

            logits_taste = taste.detach().cpu().numpy()
            logits_quantity = quantity.detach().cpu().numpy()
            logits_delivery = delivery.detach().cpu().numpy()

            print('맛에 대한 점수:',logits_taste)
            print('양 or 서비스에 대한 점수:',logits_quantity)
            print('배달에 대한 점수:',logits_delivery)

---
# 결과값 테스트
---

In [31]:
import warnings
warnings.filterwarnings('ignore')

In [32]:
predict('맛이 없다')

맛에 대한 점수: [0.5902604]
양 or 서비스에 대한 점수: [0.1711593]
배달에 대한 점수: [0.13030398]


In [26]:
predict('서비스가 좋아서 다시 가고 싶어요')

맛에 대한 점수: [0.6399003]
양 or 서비스에 대한 점수: [0.19972572]
배달에 대한 점수: [0.24509546]


In [32]:
predict('돈가스 맛집이라해서 가봤는데 양이 너무 적어요')

맛에 대한 점수: [0.55366737]
양 or 서비스에 대한 점수: [0.20569868]
배달에 대한 점수: [0.17289019]


In [ ]:
predict('음식은 겉바속촉 매장은 깔끔')

맛에 대한 점수: [0.13244781]
양 or 서비스에 대한 점수: [0.01129058]
배달에 대한 점수: [0.00538063]


In [ ]:
predict('가까워서 빠른 배달이 좋아요 상하이 맛있구요')

맛에 대한 점수: [0.9810798]
양 or 서비스에 대한 점수: [0.01186099]
배달에 대한 점수: [0.97552365]


In [ ]:
predict('맛 양 배달 모두 만족합니다')

맛에 대한 점수: [0.9852116]
양 or 서비스에 대한 점수: [0.9312223]
배달에 대한 점수: [0.9521803]


In [ ]:
predict('생각보다 빨리 와서 좋았어요 ㅎㅎ')

맛에 대한 점수: [0.01635768]
양 or 서비스에 대한 점수: [0.01938385]
배달에 대한 점수: [0.94592196]


In [ ]:
predict('다 좋은데 피자가 찌그러져서 왔네요')

맛에 대한 점수: [0.00905802]
양 or 서비스에 대한 점수: [0.01355946]
배달에 대한 점수: [0.0275925]


In [ ]:
predict('사업 번창하세요')

맛에 대한 점수: [0.0282548]
양 or 서비스에 대한 점수: [0.01747095]
배달에 대한 점수: [0.00746342]


In [ ]:
predict('콜라가 거의 다 넘쳤어요 조금 흐른 정도가 아니라 쏟아진 수준이라 감튀도 축축해지고 ㅜㅜ')

맛에 대한 점수: [0.01591356]
양 or 서비스에 대한 점수: [0.0123054]
배달에 대한 점수: [0.01672905]


In [ ]:
predict('기사님 너무 친절하시고 감자도 꾹꾹 담아서 가득 주셔서 배부르게 먹었어요 콜라도 뚜껑 이중으로 해주셔서 하나도 안 새고 왔습니당')

맛에 대한 점수: [0.01854088]
양 or 서비스에 대한 점수: [0.9913953]
배달에 대한 점수: [0.02620982]


In [ ]:
predict('다른집 들렸다 온것 같아요')

맛에 대한 점수: [0.01101383]
양 or 서비스에 대한 점수: [0.01098296]
배달에 대한 점수: [0.01932534]
